# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [1]:
activity=4088494919
date='September 20, 2020'
LAST=True

In [2]:
import helium
import pandas as pd
import os
import time
import sys
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
bkp=pd.read_json(db_file)
if not LAST and not bkp[bkp.Info.astype(str).str.contains(date)].empty:
    sys.exit('Check if already exists in bkp')

In [4]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [5]:
sgmt

,Name,Time,Speed,Power,Info
0,Puente - Loma Suramerica 1.06km 46m 4.0%,4:43,13.6km/h,191W,"{'Calories': {'Avg': '1879', 'Max': '1879'}, '..."
1,Subida desde la glorieta sur america 1.34km ...,9:01,9.0km/h,217W,"{'Calories': {'Avg': '1879', 'Max': '1879'}, '..."
2,Loma Mall Suramerica 0.31km 17m 4.2%,1:59,9.6km/h,192W,"{'Calories': {'Avg': '1879', 'Max': '1879'}, '..."
3,El concejo Prado 3.39km 145m 4.3%,21:43,9.4km/h,205W,"{'Calories': {'Avg': '1879', 'Max': '1879'}, '..."
4,13 % 0.35km 47m 13.4%,4:26,4.8km/h,200W,"{'Calories': {'Avg': '1879', 'Max': '1879'}, '..."
5,Calle 3 Climb 0.51km 49m 9.5%,4:52,6.3km/h,203W,"{'Calories': {'Avg': '1879', 'Max': '1879'}, '..."
6,Carrera 6 Climb 0.40km 23m 5.7%,2:23,10.2km/h,258W,"{'Calories': {'Avg': '1879', 'Max': '1879'}, '..."
7,Remate al Parque SAP 0.59km 35m 5.1%,6:56,5.1km/h,119W,"{'Calories': {'Avg': '1879', 'Max': '1879'}, '..."
8,Prado - Partidas 5.05km 284m 5.5%,42:48,7.1km/h,163W,"{'Calories': {'Avg': '1879', 'Max': '1879'}, '..."
9,Prado - Destapado 2.69km 130m 4.5%,17:03,9.5km/h,198W,"{'Calories': {'Avg': '1879', 'Max': '1879'}, '..."


In [6]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [7]:
df.to_json(db_file)

### Save notebook here

In [8]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [9]:
!bash commit.sh

[master fc62460] Morning Ride 8:32 AM on Monday, October 12, 2020
 2 files changed, 439 insertions(+), 41 deletions(-)
 rewrite db/strava.json (67%)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 21.86 KiB | 2.43 MiB/s, done.
Total 5 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:restrepo/strava.git
   97ac5eb..fc62460  master -> master


In [ ]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()